In [22]:
import pandas as pd
import os

# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    #filling the missing data
    total_missing = raw_data.isna().sum(axis=0)
    misssing_value_columns= list(total_missing[total_missing > 0].index)
    # Write your code here
    for missing in misssing_value_columns:
        raw_data[missing] = raw_data[missing].fillna(raw_data[missing].mode()[0])

    # convert Date series to datime format "%Y-%m-%d"
    # format and extract month in 'Date' field
    raw_data['Date'] = pd.to_datetime(raw_data['Date'], format="%Y-%m-%d")
    raw_data['Month'] = raw_data['Date'].dt.month.astype('int32')

    # keeping the rows where the weekly sales are over $10,000 and dropping the unnecessary columns
    raw_data = raw_data[raw_data['Weekly_Sales'] > 10000]

    # list of the necessary columns
    nec_columns = ['Store_ID', 'Weekly_Sales', 'IsHoliday', 'Month', 'CPI', 'Unemployment']
    # list all column
    columns = list(raw_data.columns)
    # drop unnecessary columns
    for column in columns:
        if column not in nec_columns:
            raw_data = raw_data.drop(column, axis=1)
    
    return raw_data

In [23]:
grocery_sales = pd.read_csv("grocery_sales.csv")
print(grocery_sales.head())

   Unnamed: 0  index  Store_ID        Date  Dept  Weekly_Sales  IsHoliday  \
0           0      0         1  2010-02-05     1      24924.50          0   
1           1      1         1  2010-02-05    26      11737.12          0   
2           2      2         1  2010-02-05    17      13223.76          0   
3           3      3         1  2010-02-05    45         37.44          0   
4           4      4         1  2010-02-05    28       1085.29          0   

   Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0        42.31       2.572        0.0        0.0        0.0        0.0   
1        42.31       2.572        0.0        0.0        0.0        0.0   
2        42.31       2.572        0.0        0.0        0.0        0.0   
3        42.31       2.572        0.0        0.0        0.0        0.0   
4        42.31       2.572        0.0        0.0        0.0        0.0   

   MarkDown5         CPI  Unemployment  Type      Size  
0        0.0  211.096358         8.

In [ ]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(grocery_sales)

# check whether there is any missing data.
assert clean_data.isna().any().any() == False
# check Values of Weekly sale > 10000$
assert (clean_data['Weekly_Sales']>10000).any()
# check added 'Month' field
assert 'Month' in list(clean_data.columns)

In [25]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    mw = clean_data[['Month', 'Weekly_Sales']]
    Avg_Sales = mw.groupby('Month')['Weekly_Sales'].mean()
    agg = Avg_Sales.round(2)
    agg = agg.reset_index()
    agg.rename(columns={"Weekly_Sales": "Avg_Sales"}, inplace=True)
    return agg

In [26]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
print(agg_data)

    Month  Avg_Sales
0       1   33174.18
1       2   34331.29
2       3   33227.31
3       4   33414.78
4       5   33339.89
5       6   34582.47
6       7   33930.77
7       8   33644.79
8       9   33266.59
9      10   32736.99
10     11   36594.03
11     12   39248.98


In [27]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)
    pass

In [28]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
load(clean_data, "clean_data.csv", agg_data, "agg_data.csv")

In [29]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    if not os.path.exists(file_path):
        raise Exception(f"File not found: {file_path}")

    print("File exists!")

In [30]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("clean_data.csv")
validation("agg_data.csv")

File exists!
File exists!
